In [28]:
import pandas as pd

data = pd.read_csv('./Market_Basket_Optimisation.csv',header=None)


In [188]:
pd.options.display.max_columns=100
data.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,shrimp,almonds,avocado,vegetables mix,green grapes,whole weat flour,yams,cottage cheese,energy drink,tomato juice,low fat yogurt,green tea,honey,salad,mineral water,salmon,antioxydant juice,frozen smoothie,spinach,olive oil
1,burgers,meatballs,eggs,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,chutney,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,turkey,avocado,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,mineral water,milk,energy bar,whole wheat rice,green tea,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [189]:
#efficient_apriori方法
from efficient_apriori import apriori
transactions=[]
for i in range(len(data)):
    temp_set=set()
    temp_transactions = data.iloc[i].dropna()
    for j in range(len(temp_transactions)):
        temp_set.add(temp_transactions[j])
    transactions.append(temp_set)

In [47]:
itemsets,rules = apriori(transactions,min_support=0.05,min_confidence=0.2)
print('频繁项集：',itemsets)
print('关联规则：',rules)

频繁项集： {1: {('burgers',): 654, ('cake',): 608, ('chicken',): 450, ('chocolate',): 1229, ('cookies',): 603, ('cooking oil',): 383, ('eggs',): 1348, ('escalope',): 595, ('french fries',): 1282, ('frozen smoothie',): 475, ('frozen vegetables',): 715, ('grated cheese',): 393, ('green tea',): 991, ('ground beef',): 737, ('low fat yogurt',): 574, ('milk',): 972, ('mineral water',): 1788, ('olive oil',): 494, ('pancakes',): 713, ('shrimp',): 536, ('soup',): 379, ('spaghetti',): 1306, ('tomatoes',): 513, ('turkey',): 469, ('whole wheat rice',): 439}, 2: {('chocolate', 'mineral water'): 395, ('eggs', 'mineral water'): 382, ('mineral water', 'spaghetti'): 448}}
关联规则： [{mineral water} -> {chocolate}, {chocolate} -> {mineral water}, {mineral water} -> {eggs}, {eggs} -> {mineral water}, {spaghetti} -> {mineral water}, {mineral water} -> {spaghetti}]


In [ ]:
由上述结果可以看到，最受欢迎的商品组合包括('chocolate', 'mineral water')，
                      ('mineral water', 'spaghetti')，
                     ('eggs', 'mineral water')，
购买chocolate，egg，spaghetti都有助于提升mineral water的销量。

In [169]:
#进一步采用mlxtend方法分析。
import pandas as pd
import numpy as np
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
data = pd.read_csv('./Market_Basket_Optimisation.csv',header=None)
#one-hot编码预处理
transactions=[]
for i in range(len(data)):
    temp_set=set()
    temp_transactions = data.iloc[i].dropna()
    for j in range(len(temp_transactions)):
        temp_set.add(temp_transactions[j])
    transactions.append(temp_set)
transaction_item = []
for num_transaction in range(len(transactions)):
    list_items=list(transactions[num_transaction])
    result=''
    for num_item in range(len(list_items)):
        result+=list_items[num_item]+'|'
    transaction_item.append(result)
    #print(result)

new_data = {'item':transaction_item}

df=pd.DataFrame(new_data)
df.to_csv('new_data.csv')

In [181]:
market= pd.read_csv('./new_data.csv')
market_hot_encoded = market.join(market.item.str.get_dummies(sep='|')).drop('item',axis=1)
market_hot_encoded.head()
itemsets=apriori(market_hot_encoded,use_colnames=True,min_support=0.05)
itemsets = itemsets.sort_values(by='support',ascending=False)
print('-'*20,'频繁项集','-'*20)
print(itemsets)

-------------------- 频繁项集 --------------------
     support                    itemsets
16  0.238368             (mineral water)
6   0.179709                      (eggs)
21  0.174110                 (spaghetti)
8   0.170911              (french fries)
3   0.163845                 (chocolate)
12  0.132116                 (green tea)
15  0.129583                      (milk)
13  0.098254               (ground beef)
10  0.095321         (frozen vegetables)
18  0.095054                  (pancakes)
0   0.087188                   (burgers)
1   0.081056                      (cake)
4   0.080389                   (cookies)
7   0.079323                  (escalope)
14  0.076523            (low fat yogurt)
19  0.071457                    (shrimp)
22  0.068391                  (tomatoes)
17  0.065858                 (olive oil)
9   0.063325           (frozen smoothie)
23  0.062525                    (turkey)
2   0.059992                   (chicken)
27  0.059725  (mineral water, spaghetti)
24  0.0585

In [186]:
rules = association_rules(itemsets,metric='lift',min_threshold=0.5)
rules = rules.sort_values(by='lift',ascending=False)
print('-'*20,'关联规则','-'*20)
print(rules)

-------------------- 关联规则 --------------------
       antecedents      consequents  antecedent support  consequent support  \
0  (mineral water)      (spaghetti)            0.238368            0.174110   
1      (spaghetti)  (mineral water)            0.174110            0.238368   
3      (chocolate)  (mineral water)            0.163845            0.238368   
2  (mineral water)      (chocolate)            0.238368            0.163845   
4  (mineral water)           (eggs)            0.238368            0.179709   
5           (eggs)  (mineral water)            0.179709            0.238368   

    support  confidence      lift  leverage  conviction  
0  0.059725    0.250559  1.439085  0.018223    1.102008  
1  0.059725    0.343032  1.439085  0.018223    1.159314  
3  0.052660    0.321400  1.348332  0.013604    1.122357  
2  0.052660    0.220917  1.348332  0.013604    1.073256  
4  0.050927    0.213647  1.188845  0.008090    1.043158  
5  0.050927    0.283383  1.188845  0.008090    1.06